# Tuberculosis Diagnosis using Transfer Learning
This notebook is written and executed by **Dr Raheel Siddiqi** on *06-10-2019*. The notebook presents an experiment to classify X-ray images as 'NORMAL' or 'containing manifestation of Tuberculosis (TB)' i.e. it is a binary classification problem. Transfer Learning is used to exploit the feature extractor of the *VGG16* pre-trained model. The dataset used is: [**China Set - The Shenzhen set - Chest X-ray Database**](https://www.kaggle.com/kmader/pulmonary-chest-xray-abnormalities). 5-fold cross validation is used to evaluate the model.

In [1]:
import tensorflow as tf
from tensorflow.python import keras

print('Tensorflow Version: ', tf.__version__)
print('Keras Version: ', keras.__version__)

Tensorflow Version:  1.13.1
Keras Version:  2.2.4-tf


In [2]:
from tensorflow.python.keras.applications import VGG16
import os
import numpy as np
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from tensorflow.python.keras import optimizers

def get_model():
    model = models.Sequential()
    conv_base=VGG16(weights='imagenet',include_top=False,input_shape=(100,100,3))
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=optimizers.Adam(lr=1e-4),metrics=['accuracy'])
    return model   

In [3]:
image_height = 100
image_width = 100
batch_size = 2
no_of_epochs  = 15

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
%matplotlib inline

DATADIR = "D:\\TensorFlow Programs\\TB Diagnosis\\ChinaSet_AllFiles\\ChinaSet_AllFiles\\ChinaSetImages"

CATEGORIES = ["NORMAL", "TB"]

data=[]

for category in CATEGORIES:  
    path = os.path.join(DATADIR,category)
    class_num=CATEGORIES.index(category)
    for img in tqdm(os.listdir(path)):  
        try:
            img_array = cv2.imread(os.path.join(path,img))
            img_array = cv2.resize(img_array, (image_height, image_width))
            img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)        
            img_array = img_array.astype(np.float32)/255.
            data.append([img_array, class_num])
        except Exception as e:   
            pass

print(len(data))

100%|████████████████████████████████████████████████████████████████████████████████| 336/336 [00:35<00:00,  7.73it/s]


662


In [5]:
import random

random.shuffle(data)
for sample in data[:10]:
    print(sample[1])

0
1
1
0
1
1
0
1
0
1


In [6]:
X = []
y = []

for features,label in data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, image_width, image_height, 3)
print(X.shape)

(662, 100, 100, 3)


In [7]:
k=5
num_validation_samples=len(X)//k
validation_scores=[]
for fold in range(k):
    validation_data=X[num_validation_samples*fold:num_validation_samples*(fold+1)]
    validation_labels=y[num_validation_samples*fold:num_validation_samples*(fold+1)]
    if fold==0:
        training_data=X[num_validation_samples*(fold+1):]
        training_labels=y[num_validation_samples*(fold+1):]    
    else:
        training_data=np.append(X[:num_validation_samples*fold], X[num_validation_samples*(fold+1):],axis=0)
        training_labels=np.append(y[:num_validation_samples*fold], y[num_validation_samples*(fold+1):],axis=0)
    model=get_model()
    model.fit(training_data,training_labels,batch_size=4,epochs=15) # 15 epochs per model
    validation_score=model.evaluate(validation_data,validation_labels)
    validation_scores.append(validation_score[1])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/15
530/530 [==============================] - ETA: 7:40 - loss: 1.1938 - acc: 0.0000e+0 - ETA: 3:54 - loss: 1.0626 - acc: 0.1250    - ETA: 2:39 - loss: 0.9679 - acc: 0.166 - ETA: 2:01 - loss: 0.8886 - acc: 0.312 - ETA: 1:38 - loss: 0.8127 - acc: 0.450 - ETA: 1:24 - loss: 0.8082 - acc: 0.458 - ETA: 1:13 - loss: 0.8055 - acc: 0.464 - ETA: 1:04 - loss: 0.8101 - acc: 0.437 - ETA: 58s - loss: 0.7922 - acc: 0.500 - ETA: 53s - loss: 0.7691 - acc: 0.52 - ETA: 49s - loss: 0.8183 - acc: 0.50 - ETA: 45s - loss: 0.8145 - acc: 0.50 - ETA: 42s - loss: 0.8034 - acc: 0.53 - ETA: 40s - loss: 0.7967 - acc: 0.53 - ETA: 37s - loss: 0.7944 - acc: 0.51 - ETA: 35s - loss: 0.7873 - acc: 0.51 - ETA: 34s - loss: 0.7780 - acc: 0.52 - ETA: 32s - loss: 0.7734 - acc: 0.52 - ETA: 31s - loss: 0.7656 - acc: 0.53 - ETA: 29s - loss: 0.7569 - acc: 0.55 - ETA: 28s - loss: 0.7464 - acc: 0.55 - ETA

530/530 [==============================] - ETA: 12s - loss: 0.4805 - acc: 0.75 - ETA: 12s - loss: 0.5166 - acc: 0.62 - ETA: 12s - loss: 0.3968 - acc: 0.75 - ETA: 12s - loss: 0.4426 - acc: 0.75 - ETA: 11s - loss: 0.4452 - acc: 0.80 - ETA: 11s - loss: 0.4054 - acc: 0.83 - ETA: 11s - loss: 0.3690 - acc: 0.85 - ETA: 11s - loss: 0.4144 - acc: 0.84 - ETA: 11s - loss: 0.3862 - acc: 0.86 - ETA: 11s - loss: 0.4223 - acc: 0.85 - ETA: 11s - loss: 0.3940 - acc: 0.86 - ETA: 11s - loss: 0.3910 - acc: 0.85 - ETA: 11s - loss: 0.4572 - acc: 0.82 - ETA: 11s - loss: 0.4905 - acc: 0.80 - ETA: 11s - loss: 0.4914 - acc: 0.80 - ETA: 10s - loss: 0.5020 - acc: 0.79 - ETA: 10s - loss: 0.5004 - acc: 0.79 - ETA: 10s - loss: 0.4909 - acc: 0.80 - ETA: 10s - loss: 0.4805 - acc: 0.81 - ETA: 10s - loss: 0.4841 - acc: 0.81 - ETA: 10s - loss: 0.4819 - acc: 0.80 - ETA: 10s - loss: 0.4883 - acc: 0.80 - ETA: 10s - loss: 0.4935 - acc: 0.80 - ETA: 10s - loss: 0.4981 - acc: 0.80 - ETA: 10s - loss: 0.5193 - acc: 0.79 - ETA: 10

530/530 [==============================] - ETA: 12s - loss: 0.6527 - acc: 0.50 - ETA: 12s - loss: 0.4302 - acc: 0.75 - ETA: 12s - loss: 0.3940 - acc: 0.83 - ETA: 11s - loss: 0.3180 - acc: 0.87 - ETA: 11s - loss: 0.3517 - acc: 0.85 - ETA: 11s - loss: 0.3236 - acc: 0.87 - ETA: 11s - loss: 0.3139 - acc: 0.85 - ETA: 11s - loss: 0.2921 - acc: 0.87 - ETA: 11s - loss: 0.3548 - acc: 0.86 - ETA: 11s - loss: 0.3855 - acc: 0.85 - ETA: 11s - loss: 0.3674 - acc: 0.86 - ETA: 11s - loss: 0.3652 - acc: 0.85 - ETA: 11s - loss: 0.3387 - acc: 0.86 - ETA: 11s - loss: 0.3159 - acc: 0.87 - ETA: 11s - loss: 0.4039 - acc: 0.86 - ETA: 10s - loss: 0.4137 - acc: 0.85 - ETA: 10s - loss: 0.4466 - acc: 0.83 - ETA: 10s - loss: 0.4356 - acc: 0.83 - ETA: 10s - loss: 0.4349 - acc: 0.84 - ETA: 10s - loss: 0.4417 - acc: 0.83 - ETA: 10s - loss: 0.4747 - acc: 0.82 - ETA: 10s - loss: 0.4648 - acc: 0.82 - ETA: 10s - loss: 0.4561 - acc: 0.83 - ETA: 10s - loss: 0.4402 - acc: 0.84 - ETA: 10s - loss: 0.4317 - acc: 0.85 - ETA: 10

530/530 [==============================] - ETA: 12s - loss: 0.3801 - acc: 1.00 - ETA: 12s - loss: 0.4432 - acc: 0.87 - ETA: 12s - loss: 0.4249 - acc: 0.91 - ETA: 11s - loss: 0.3852 - acc: 0.93 - ETA: 11s - loss: 0.3569 - acc: 0.95 - ETA: 11s - loss: 0.3273 - acc: 0.95 - ETA: 11s - loss: 0.3104 - acc: 0.96 - ETA: 11s - loss: 0.3605 - acc: 0.93 - ETA: 11s - loss: 0.3741 - acc: 0.91 - ETA: 11s - loss: 0.3537 - acc: 0.92 - ETA: 11s - loss: 0.3683 - acc: 0.90 - ETA: 11s - loss: 0.3804 - acc: 0.89 - ETA: 11s - loss: 0.3604 - acc: 0.90 - ETA: 11s - loss: 0.3435 - acc: 0.91 - ETA: 11s - loss: 0.3306 - acc: 0.91 - ETA: 10s - loss: 0.4025 - acc: 0.89 - ETA: 10s - loss: 0.3839 - acc: 0.89 - ETA: 10s - loss: 0.4243 - acc: 0.87 - ETA: 10s - loss: 0.4414 - acc: 0.85 - ETA: 10s - loss: 0.4361 - acc: 0.85 - ETA: 10s - loss: 0.4594 - acc: 0.83 - ETA: 10s - loss: 0.4515 - acc: 0.84 - ETA: 10s - loss: 0.4519 - acc: 0.83 - ETA: 10s - loss: 0.4664 - acc: 0.83 - ETA: 10s - loss: 0.4698 - acc: 0.83 - ETA: 9s

530/530 [==============================] - ETA: 11s - loss: 0.2083 - acc: 1.00 - ETA: 11s - loss: 0.5545 - acc: 0.75 - ETA: 11s - loss: 0.5489 - acc: 0.75 - ETA: 11s - loss: 0.4393 - acc: 0.81 - ETA: 11s - loss: 0.5050 - acc: 0.70 - ETA: 11s - loss: 0.4417 - acc: 0.75 - ETA: 11s - loss: 0.4026 - acc: 0.78 - ETA: 11s - loss: 0.3698 - acc: 0.81 - ETA: 11s - loss: 0.3369 - acc: 0.83 - ETA: 11s - loss: 0.3453 - acc: 0.82 - ETA: 11s - loss: 0.4087 - acc: 0.79 - ETA: 11s - loss: 0.3780 - acc: 0.81 - ETA: 11s - loss: 0.4007 - acc: 0.78 - ETA: 11s - loss: 0.4014 - acc: 0.78 - ETA: 10s - loss: 0.4268 - acc: 0.78 - ETA: 10s - loss: 0.4266 - acc: 0.78 - ETA: 10s - loss: 0.4167 - acc: 0.79 - ETA: 10s - loss: 0.4216 - acc: 0.79 - ETA: 10s - loss: 0.4092 - acc: 0.80 - ETA: 10s - loss: 0.4016 - acc: 0.81 - ETA: 10s - loss: 0.3891 - acc: 0.82 - ETA: 10s - loss: 0.3826 - acc: 0.81 - ETA: 10s - loss: 0.3766 - acc: 0.82 - ETA: 10s - loss: 0.3840 - acc: 0.82 - ETA: 10s - loss: 0.3740 - acc: 0.83 - ETA: 9s

...

530/530 [==============================] - ETA: 11s - loss: 0.1812 - acc: 1.00 - ETA: 12s - loss: 0.1433 - acc: 1.00 - ETA: 12s - loss: 0.2018 - acc: 0.91 - ETA: 11s - loss: 0.1949 - acc: 0.93 - ETA: 11s - loss: 0.1914 - acc: 0.95 - ETA: 11s - loss: 0.1741 - acc: 0.95 - ETA: 11s - loss: 0.2036 - acc: 0.92 - ETA: 11s - loss: 0.2216 - acc: 0.90 - ETA: 11s - loss: 0.2231 - acc: 0.91 - ETA: 11s - loss: 0.2354 - acc: 0.90 - ETA: 11s - loss: 0.2499 - acc: 0.88 - ETA: 11s - loss: 0.2747 - acc: 0.87 - ETA: 11s - loss: 0.3136 - acc: 0.84 - ETA: 11s - loss: 0.3623 - acc: 0.83 - ETA: 10s - loss: 0.3821 - acc: 0.83 - ETA: 10s - loss: 0.3700 - acc: 0.84 - ETA: 10s - loss: 0.4038 - acc: 0.82 - ETA: 10s - loss: 0.4734 - acc: 0.79 - ETA: 10s - loss: 0.4677 - acc: 0.80 - ETA: 10s - loss: 0.4584 - acc: 0.81 - ETA: 10s - loss: 0.4447 - acc: 0.82 - ETA: 10s - loss: 0.4385 - acc: 0.81 - ETA: 10s - loss: 0.4541 - acc: 0.80 - ETA: 10s - loss: 0.4430 - acc: 0.81 - ETA: 10s - loss: 0.4430 - acc: 0.81 - ETA: 9s

530/530 [==============================] - ETA: 10s - loss: 0.1813 - acc: 1.00 - ETA: 11s - loss: 0.2166 - acc: 1.00 - ETA: 11s - loss: 0.3047 - acc: 0.91 - ETA: 11s - loss: 0.2567 - acc: 0.93 - ETA: 11s - loss: 0.2335 - acc: 0.95 - ETA: 11s - loss: 0.2106 - acc: 0.95 - ETA: 11s - loss: 0.2015 - acc: 0.96 - ETA: 11s - loss: 0.3014 - acc: 0.90 - ETA: 11s - loss: 0.2818 - acc: 0.91 - ETA: 11s - loss: 0.3060 - acc: 0.90 - ETA: 11s - loss: 0.3286 - acc: 0.86 - ETA: 11s - loss: 0.3464 - acc: 0.85 - ETA: 11s - loss: 0.3339 - acc: 0.86 - ETA: 11s - loss: 0.3392 - acc: 0.85 - ETA: 10s - loss: 0.3291 - acc: 0.86 - ETA: 10s - loss: 0.3145 - acc: 0.87 - ETA: 10s - loss: 0.3032 - acc: 0.88 - ETA: 10s - loss: 0.2908 - acc: 0.88 - ETA: 10s - loss: 0.3540 - acc: 0.88 - ETA: 10s - loss: 0.3958 - acc: 0.85 - ETA: 10s - loss: 0.3825 - acc: 0.85 - ETA: 10s - loss: 0.3741 - acc: 0.86 - ETA: 10s - loss: 0.3715 - acc: 0.85 - ETA: 10s - loss: 0.3598 - acc: 0.86 - ETA: 10s - loss: 0.3457 - acc: 0.87 - ETA: 9s

530/530 [==============================] - ETA: 11s - loss: 0.1429 - acc: 1.00 - ETA: 12s - loss: 0.1754 - acc: 1.00 - ETA: 12s - loss: 0.1847 - acc: 1.00 - ETA: 11s - loss: 0.2426 - acc: 0.93 - ETA: 11s - loss: 0.2033 - acc: 0.95 - ETA: 11s - loss: 0.4721 - acc: 0.91 - ETA: 11s - loss: 0.4672 - acc: 0.89 - ETA: 11s - loss: 0.4184 - acc: 0.90 - ETA: 11s - loss: 0.4101 - acc: 0.88 - ETA: 11s - loss: 0.4149 - acc: 0.87 - ETA: 11s - loss: 0.4109 - acc: 0.86 - ETA: 11s - loss: 0.4090 - acc: 0.85 - ETA: 11s - loss: 0.3883 - acc: 0.86 - ETA: 11s - loss: 0.3918 - acc: 0.85 - ETA: 11s - loss: 0.3920 - acc: 0.85 - ETA: 10s - loss: 0.4157 - acc: 0.84 - ETA: 10s - loss: 0.3972 - acc: 0.85 - ETA: 10s - loss: 0.3946 - acc: 0.84 - ETA: 10s - loss: 0.3887 - acc: 0.85 - ETA: 10s - loss: 0.3740 - acc: 0.86 - ETA: 10s - loss: 0.3624 - acc: 0.86 - ETA: 10s - loss: 0.3899 - acc: 0.85 - ETA: 10s - loss: 0.4574 - acc: 0.82 - ETA: 10s - loss: 0.4586 - acc: 0.82 - ETA: 10s - loss: 0.4548 - acc: 0.82 - ETA: 10

530/530 [==============================] - ETA: 12s - loss: 0.0893 - acc: 1.00 - ETA: 12s - loss: 0.1012 - acc: 1.00 - ETA: 12s - loss: 0.1779 - acc: 0.91 - ETA: 11s - loss: 0.4756 - acc: 0.81 - ETA: 11s - loss: 0.4091 - acc: 0.85 - ETA: 11s - loss: 0.3693 - acc: 0.87 - ETA: 11s - loss: 0.3462 - acc: 0.89 - ETA: 11s - loss: 0.3140 - acc: 0.90 - ETA: 11s - loss: 0.3421 - acc: 0.88 - ETA: 11s - loss: 0.3552 - acc: 0.87 - ETA: 11s - loss: 0.3282 - acc: 0.88 - ETA: 11s - loss: 0.3102 - acc: 0.89 - ETA: 11s - loss: 0.3019 - acc: 0.90 - ETA: 11s - loss: 0.2985 - acc: 0.91 - ETA: 11s - loss: 0.2853 - acc: 0.91 - ETA: 10s - loss: 0.2705 - acc: 0.92 - ETA: 10s - loss: 0.2944 - acc: 0.91 - ETA: 10s - loss: 0.3172 - acc: 0.90 - ETA: 10s - loss: 0.3161 - acc: 0.89 - ETA: 10s - loss: 0.3038 - acc: 0.90 - ETA: 10s - loss: 0.2995 - acc: 0.90 - ETA: 10s - loss: 0.2876 - acc: 0.90 - ETA: 10s - loss: 0.2783 - acc: 0.91 - ETA: 10s - loss: 0.2895 - acc: 0.90 - ETA: 10s - loss: 0.2944 - acc: 0.90 - ETA: 10

132/132 [==============================] - ETA: 1s - loss: 0.4550 - acc: 0.812 - ETA: 0s - loss: 0.4212 - acc: 0.859 - ETA: 0s - loss: 0.3634 - acc: 0.875 - ETA: 0s - loss: 0.3440 - acc: 0.875 - 1s 5ms/sample - loss: 0.3444 - acc: 0.8712


In [8]:
print('Average Validation Score: ', np.average(validation_scores))

Average Validation Score:  0.8439394
